In [10]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 24 17:52:34 2022

@author: asutter-adc
"""

#nouvelle version des fonctions d'identification et de recherche de siren, prends en compte les siren attachés. Même process de recherche que test4.


from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from re import compile



from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from re import compile

def is_luhn_valid(x: int) -> bool:
    """
    Application de la formule de Luhn à un nombre
    Permet la verification du numero SIREN et Siret d'un acheteur/etablissement
    Retour:
        - bool
    """
    try:
        luhn_corr = [0, 2, 4, 6, 8, 1, 3, 5, 7, 9]
        list_number_in_x = [int(i) for i in list(str(x))]
        l2 = [luhn_corr[i] if (index + 1) % 2 == 0 else i for index, i in enumerate(list_number_in_x[::-1])]
        if sum(l2) % 10 == 0:
            return True
        elif str(x)[:9] == "356000000":  # SIREN de la Poste
            if sum(list_number_in_x) % 5 == 0:
                return True
        return False
    except:
        return False



def isSIREN(c) :
    if len(c) == 11 :
        if c[0:3].isdigit() and c[3]==' ' and c[4:7].isdigit() and c[7]==' ' and c[8:].isdigit() :
            return is_luhn_valid(''.join(c.split()))

    elif len(c) == 9 :
        return is_luhn_valid(c)

    return False

def findSIREN(chaine):

    if len(chaine) < 14 :
        return 'Pas de SIREN trouvé'

    else :
        #première étape : on recherche dans le document un SIREN sous le type ??? ??? ???
        k=0

        while k <= len(chaine)-11 :
            if isSIREN(chaine[k:k+11]) :
                return chaine[k:k+11]

            k+=1

        #deuxième étape : on recherche un numéro de TVA au format FR??SIREN
        k= 0

        while k<=len(chaine) -13 :
            if chaine[k:k+2]=='FR' and chaine[k+2:k+13].isdigit() :
                if is_luhn_valid(chaine[k+4:k+13]):
                    return chaine[k+4:k+13]

            k+=1

        #troisième étape : on recherche un SIRET sans espace dont on extrait le SIREN, avec un espace le précédant (essentiel pour éviter les longues suites de chiffres présents dans les liens)
        k=0

        while k <= len(chaine)-16 :
            if chaine[k+1:k+15].isdigit() and chaine[k]==' ':
                if is_luhn_valid(chaine[k+1:k+15]) and is_luhn_valid(chaine[k+1:k+10]) :
                    return chaine[k+1:k+10]

            k+=1

        #quatrième étape : on cherche un SIREN tout attaché avec espaces avant et après
        k=0

        while k<= len(chaine)-11 :

            if chaine[k]==' ' and chaine[k+1:k+10].isdigit() and chaine[k+10]==' ':
                if is_luhn_valid(chaine[k+1:k+10]) :
                    return chaine[k+1:k+10]

            elif chaine[k]==' ' and chaine[k+1:k+10].isdigit() and chaine[k+10]=='.':
                if is_luhn_valid(chaine[k+1:k+10]) :
                    return chaine[k+1:k+10]

            k+=1


    return 'Pas de SIREN trouvé'




lstpages = ['Mentions légales', 'Mentions Légales', 'mentions légales', 'Legal Notice', 'Legal Notices', 'Legal notice', 'Legal notices', 'Legal', 'Legals', 'legal', 'legals', 'Terms', 'terms', 'Conditions Générales de Vente', 'Conditions générales de vente', 'Conditions Générales', 'Conditions générales', 'conditions générales' ,'CG', 'Conditions générales', 'Conditions de vente', 'Conditions de Vente', 'Mentions légales & CGU', 'CGUV', 'CGU', 'Terms of use', 'Terms of Use', 'Legal Information', 'Legal information', 'Legal Informations', 'Legal informations', 'Privacy', 'privacy', 'Privacy Policy', 'Privacy policy', 'Terms of service', 'Legal Mentions', 'Legal mentions', 'legal mentions', 'Legal Terms', 'Legal terms', 'About Us', 'About us', 'Disclaimer']



nom = 'sitesDR2001.csv'
siren = 'nomsirenDR2001.txt'

file = open(nom, encoding = 'utf-8')
newfile = open(siren, 'w', encoding = 'utf-8')
i=1

for ligne in file :
    print(i)
    lst = ligne.strip().split(';')
    url = lst[0]
    try :
        if url !='' :

            if url[-1]=='/' :
                url=url[:-1]

            res = requests.get(url)

            soup=BeautifulSoup(res.content, 'lxml')

            k = 0
            length = len(lstpages)
            sirentrouve=False
            siren = 'Pages introuvables'
            pageouverte=False


            while k<length and not sirentrouve :

                page = lstpages[k]
                #premier test simple
                link = soup.find('a', string=page, href=True)

                if link is not None :
                    pageouverte=True
                    newurl = link.get('href')

                    if url[8:10]=='ww' :
                        test=url[12:-4]

                    else :
                        test = url[8:-4]

                    if newurl.find(test) == -1 and newurl[:2]!='ht' and newurl[:2]!='//' : #cas où url relatif
                        if url[-3:]=='/fr' and newurl[:3]=='/fr':
                            newurl = url+newurl[3:]

                        elif url[-4:]=='/fr/' and newurl[:3]=='/fr' :
                            newurl = url + newurl[4:]

                        else :
                            newurl = url + newurl

                    if newurl[:2]=='//': #cas où url non relatif mais ne commençant pas par http:
                        newurl='http:'+newurl

                    newres = requests.get(newurl)
                    contenu = str(newres.content)

                    #siren = findSIREN(contenu)

                    #sirentrouve = isSIREN(siren)

                    #if not sirentrouve : #gérons les espaces insécables

                    soup2=BeautifulSoup(newres.content, 'html.parser')
                    chaine2=str(soup2)

                    chaine3=chaine2.replace('\xa0', ' ')


                    siren = findSIREN(chaine3)
                    sirentrouve=isSIREN(siren)

                k+=1

            if not sirentrouve :

                link=None
                k=0
                while k<length and not sirentrouve :

                    page = lstpages[k]

                #on teste ceux où l'encodage a été correct mais où la string est à striper
                    if link is None :

                        liens = soup.find_all('a', href =True, string = True)
                        nbliens=len(liens)
                        compt=1
                        while compt <= nbliens and link is None :
                            tag=liens[nbliens-compt]
                            chaine=tag.string.strip()
                            if chaine.find(page)!=-1 :
                                link = tag

                            compt += 1

                #on teste ceux mal encodés, en misant sur le fait qu'ils sont encodés en latin-1
                    if link is None :

                        liens=soup.find_all('a', href=True, string=True)
                        nbliens=len(liens)
                        compt=1
                        while compt <= nbliens and link is None :
                            tag=liens[nbliens-compt]
                            chaine=str(tag.string).strip()
                            binaire= bytes(page, 'utf-8') #c'est la page qu'on décode
                            testl=binaire.decode('latin-1')

                            if chaine.find(testl)!=-1 : #on demande juste à ce que page se trouve dans string
                                link = tag

                            compt += 1



                    if link is not None :

                        newurl = link.get('href')

                        if url[8:10]=='ww' :
                            test=url[12:-4]

                        else :
                            test = url[8:-4]

                        if newurl.find(test) == -1 and newurl[:2]!='ht' and newurl[:2]!='//': #cas où url relatif
                            if url[-3:]=='/fr' and newurl[:3]=='/fr':
                                newurl = url+newurl[3:]

                            elif url[-4:]=='/fr/' and newurl[:3]=='/fr' :
                                newurl = url + newurl[4:]

                            else :
                                newurl = url + newurl

                        if newurl[:2]=='//':
                            newurl='http:'+newurl

                        newres = requests.get(newurl)
                        contenu = str(newres.content)

                        soup2=BeautifulSoup(newres.content, 'html.parser')
                        chaine2=str(soup2)
                        chaine3=chaine2.replace('\xa0', ' ')

                        siren = findSIREN(chaine3)
                        sirentrouve=isSIREN(siren)

                    k+=1

        #cas où le string contient des strings
            if not sirentrouve :

                link=None
                k=0
                while k<length and not sirentrouve :

                    page = lstpages[k]
                    if link is None :

                        liens = soup.find_all('a', href =True)
                        nbliens=len(liens)
                        compt=1
                        while compt <= nbliens and link is None :
                            tag=liens[nbliens-compt]
                            if page in tag.descendants :
                                link = tag

                            compt += 1

                    if link is None :

                        liens = soup.find_all('a', href =True)
                        nbliens=len(liens)
                        compt=1
                        while compt <= nbliens and link is None :
                            tag=liens[nbliens-compt]
                            for elem in tag.descendants :
                                if elem.string is not None :
                                    chaine = str(elem.string)
                                    if chaine.find(page)!=-1 :
                                        link = tag

                            compt += 1

                    if link is not None :

                        newurl = link.get('href')

                        if url[8:10]=='ww' :
                            test=url[12:-4]

                        else :
                            test = url[8:-4]

                        if newurl.find(test) == -1 and newurl[:2]!='ht' and newurl[:2]!='//': #cas où url relatif
                            if url[-3:]=='/fr' and newurl[:3]=='/fr':
                                newurl = url+newurl[3:]

                            elif url[-4:]=='/fr/' and newurl[:3]=='/fr' :
                                newurl = url + newurl[4:]

                            else :
                                newurl = url + newurl

                        if newurl[:2]=='//':
                            newurl='http:'+newurl

                        newres = requests.get(newurl)
                        contenu = str(newres.content)

                        #siren = findSIREN(contenu)

                        #sirentrouve = isSIREN(siren)

                        #if not sirentrouve : #gérons les espaces insécables

                        soup2=BeautifulSoup(newres.content, 'html.parser')
                        chaine2=str(soup2)
                        chaine3=chaine2.replace('\xa0', ' ')

                        siren = findSIREN(chaine3)
                        sirentrouve=isSIREN(siren)

                    k+=1
            newfile.write(str.join(';', [lst[0],siren]))
            newfile.write('\n')

    except :
        newfile.write(str.join(';', [lst[0],'erreur de connection']))
        newfile.write('\n')
    i+=1


file.close()
newfile.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [4]:
for i in range(3) :
    print(i)

0
1
2
